In [1]:
import json
from pydub import AudioSegment
from datetime import timedelta


In [2]:
INPUT_FILE = './test.mp3'
TRANSCRIPTION_DATA = './data/podcast__transcription_test_op_10.txt'
sound = AudioSegment.from_mp3("./test.mp3")


In [3]:
def read_from_file(file_path, json=True):
    with open(file_path, "r") as f:
        val = f.read()
        if json: 
            return json.loads(val)
        else: 
            return val

In [4]:
transcript_data = read_from_file(TRANSCRIPTION_DATA, json=False)
transcript_data = transcript_data.split('\n')


In [5]:
split_transcript_data = []
for i, item in enumerate(transcript_data): 
    split_data = item.split("-->")
    split_transcript_data.append(split_data)


In [6]:
def get_ms(string): 
    string = string.replace(',', ':')
    h, m, s, ms = map(float, string.split(':'))
    res = timedelta(hours=h, minutes=m, seconds=s, milliseconds=ms).total_seconds() * 1000
    return res

In [7]:
def slice_sound(start_ms, end_ms, sound_segments, filename='',  buffer = 0): 
    start_ms = start_ms - buffer
    end_ms = end_ms + buffer
    print(start_ms, end_ms)
    
    sliced_sound = sound[int(start_ms):int(end_ms)]
    silence = AudioSegment.silent(duration=1500) # or be explicit
    sound_segments.append(sliced_sound)
    sound_segments.append(silence)


In [8]:
sound_segments = []

for item in split_transcript_data: 
    if len(item) == 3: 
        start_ms = get_ms(item[0])
        end_ms = get_ms(item[1])
        print(f"Getting Sounds from: {start_ms} - {end_ms}")
        filename = item[0] + '-' + item[1] + '.mp3'
        slice_sound(start_ms, end_ms, sound_segments, filename, buffer = 200)
        


Getting Sounds from: 4160.0 - 12170.0
3960.0 12370.0
Getting Sounds from: 692400.0 - 697890.0
692200.0 698090.0
Getting Sounds from: 826240.0 - 827680.0
826040.0 827880.0
Getting Sounds from: 981875.0 - 995875.0
981675.0 996075.0
Getting Sounds from: 1403470.0 - 1406350.0
1403270.0 1406550.0
Getting Sounds from: 1812774.0 - 1815415.0
1812574.0 1815615.0
Getting Sounds from: 2162015.0 - 2166755.0
2161815.0 2166955.0
Getting Sounds from: 2216390.0 - 2219900.0
2216190.0 2220100.0
Getting Sounds from: 3364115.0 - 3377020.0
3363915.0 3377220.0
Getting Sounds from: 3705725.0 - 3710615.0
3705525.0 3710815.0


In [9]:
newSound = None
for sound in sound_segments:
    if newSound is None: 
        newSound = AudioSegment.silent(duration=500)
        newSound = newSound + sound
    else: 
        newSound = newSound + sound

In [10]:
newSound = AudioSegment.silent(duration=500) + newSound + AudioSegment.silent(duration=500)

In [11]:
newSound.export(f"./output/final.mp3", format="mp3")

<_io.BufferedRandom name='./output/final.mp3'>